In [1]:
import os
os.chdir("/Users/admin/Documents/Github/tune-track")  

In [2]:
import polars as pl
import altair as alt
from data_preparation.functions import process_original_columns
import plotly.express as px
from functools import reduce
from sklearn.cluster import KMeans

cfg = pl.Config()
cfg.set_tbl_rows(10)

polars.config.Config

In [3]:
df_sessions = pl.read_csv("data/log.csv")

df_sessions = process_original_columns(df_sessions)

## EDA

In [4]:
df_sessions.null_count()


session_id,session_position,session_length,track_id_clean,skip_1,skip_2,skip_3,not_skipped,context_switch,no_pause_before_play,short_pause_before_play,long_pause_before_play,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [4]:
df_sessions.head(1)

session_id,session_position,session_length,track_id_clean,not_skipped,context_switch,hist_user_behavior_n_seekfwd,hist_user_behavior_n_seekback,hist_user_behavior_is_shuffle,hour_of_day,date,premium,context_type,hist_user_behavior_reason_start,hist_user_behavior_reason_end,is_weekend,season,time_of_day,song_completion,pause_length_before_play
str,i64,i64,str,bool,i64,i64,i64,bool,i64,date,bool,str,str,str,i8,str,str,f64,i32
"""0_00006f66-33e5-4de7-a324-2d18…",1,20,"""t_0479f24c-27d2-46d6-a00c-7ec9…",true,0,0,0,true,16,2018-07-15,true,"""editorial_playlist""","""trackdone""","""trackdone""",1,"""summer""","""afternoon""",1.0,3


In [7]:
(df_sessions
    .group_by("context_type", "premium")
    .agg(pl.len().alias("count"))
    .sort("context_type")
)

context_type,premium,count
str,bool,u32
"""catalog""",false,6662
"""catalog""",true,31601
"""charts""",true,2134
"""charts""",false,307
"""editorial_playlist""",false,7291
"""editorial_playlist""",true,28372
"""personalized_playlist""",false,290
"""personalized_playlist""",true,2686
"""radio""",false,2662


#### Descriptive Graphs

In [8]:
import altair as alt

# Group and convert to pandas
session_counts = (
    df_sessions
    .group_by("session_length")
    .agg(pl.len().alias("count"))
    .sort("session_length")
)

# Create chart with percentage transformation
chart = alt.Chart(session_counts).transform_joinaggregate(
    TotalSessions='sum(count)'
).transform_calculate(
    Percent='datum.count / datum.TotalSessions'
).mark_bar().encode(
    x=alt.X('session_length:N', title='Session Length', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Percent:Q', title='Percentage', axis=alt.Axis(format='.0%'))
).properties(
    title="Session Length Distribution",
    width=600,
    height=400
)

chart


alt.Chart(...)

In [10]:
# Group by 'hour_of_day' and get count of each hour
hour_counts = (
    df_sessions
    .group_by("hour_of_day", "time_of_day")
    .agg(pl.len().alias("count"))
    .sort("hour_of_day")
)

# Create chart with percentage transformation
chart = alt.Chart(hour_counts).transform_joinaggregate(
    TotalSessions='sum(count)'
).transform_calculate(
    Percent='datum.count / datum.TotalSessions'
).mark_bar().encode(
    x=alt.X('hour_of_day:N', title='Hour of Day', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('Percent:Q', title='Percentage', axis=alt.Axis(format='.0%')),
    color=alt.Color('time_of_day:N', title="Time of Day", scale=alt.Scale(scheme='tableau10'))
).properties(
    title="Hour of Day Distribution",
    width=600,
    height=400
)

chart


alt.Chart(...)

## Markov Chains

In [4]:
transition_matrix = (
    df_sessions
    .group_by(["hist_user_behavior_reason_start", "hist_user_behavior_reason_end"])
    .agg(pl.len().alias("count"))
    .with_columns([
        pl.col("count")
        .sum()
        .alias("total")
    ])
    .with_columns([
        (pl.col("count") / pl.col("total")).round(3).alias("percentage")
    ])
    .pivot(
        index="hist_user_behavior_reason_start",
        on="hist_user_behavior_reason_end",
        values="percentage"
    )
)

transition_matrix


hist_user_behavior_reason_start,fwdbtn,trackdone,backbtn,logout,remote,clickrow,endplay
str,f64,f64,f64,f64,f64,f64,f64
"""backbtn""",0.014,0.018,0.042,0.0,0.0,null,0.003
"""fwdbtn""",0.366,0.07,0.022,0.001,0.0,null,0.017
"""endplay""",0.0,0.0,0.0,null,0.0,null,null
"""trackerror""",0.0,0.0,0.0,null,null,null,0.0
"""trackdone""",0.084,0.208,0.012,0.002,0.001,0.0,0.02
"""remote""",0.0,0.0,0.0,0.0,0.0,null,0.0
"""clickrow""",0.016,0.041,0.001,0.0,0.0,0.0,0.045
"""appload""",0.003,0.005,0.001,0.0,0.0,null,0.002
"""playbtn""",0.0,0.0,null,0.0,0.0,null,0.0


In [11]:
# Compute transition matrix (as row-wise normalized probabilities, rounded to 3 decimals)
transition_matrix = (
    df_sessions
    .with_columns([
        pl.col("hist_user_behavior_reason_start")
          .shift(-1)
          .over("session_id")
          .alias("hist_user_behavior_reason_start_next")
    ])
    .group_by(["hist_user_behavior_reason_start", "hist_user_behavior_reason_end"])
    .agg(pl.len().alias("count"))
    .with_columns([
        # Total outgoing transitions per start reason
        pl.col("count").sum().over("hist_user_behavior_reason_start").alias("total_start"),
        pl.col("count").sum().over("hist_user_behavior_reason_end").alias("total_end"),
        pl.col("count").sum().alias("total")
    ])
    .with_columns([
        # Normalize to percentages
        (pl.col("count") / pl.col("total")).round(3).alias("percentage")
    ])
    .pivot(
        index="hist_user_behavior_reason_start",
        on="hist_user_behavior_reason_end",
        values="percentage"
    )
)

transition_matrix


hist_user_behavior_reason_start,trackdone,endplay,fwdbtn,backbtn,logout,remote,clickrow
str,f64,f64,f64,f64,f64,f64,f64
"""trackdone""",0.208,0.02,0.084,0.012,0.002,0.001,0.0
"""endplay""",0.0,null,0.0,0.0,null,0.0,null
"""remote""",0.0,0.0,0.0,0.0,0.0,0.0,null
"""playbtn""",0.0,0.0,0.0,null,0.0,0.0,null
"""trackerror""",0.0,0.0,0.0,0.0,null,null,null
"""fwdbtn""",0.07,0.017,0.366,0.022,0.001,0.0,null
"""clickrow""",0.041,0.045,0.016,0.001,0.0,0.0,0.0
"""backbtn""",0.018,0.003,0.014,0.042,0.0,0.0,null
"""appload""",0.005,0.002,0.003,0.001,0.0,0.0,null


In [7]:
transition_stats = (
    df_sessions
    .group_by(["hist_user_behavior_reason_start", "hist_user_behavior_reason_end"])
    .agg(pl.len().alias("count"))
    .with_columns([
        # Total outgoing transitions from each start reason
        pl.col("count").sum().over("hist_user_behavior_reason_start").alias("total_reason_start"),
        
        # Total incoming transitions to each end reason
        pl.col("count").sum().over("hist_user_behavior_reason_end").alias("total_reason_end"),
        pl.col("count").sum().alias("total")
        
    ])
    .with_columns([
        # Normalize in both directions
        (pl.col("count") / pl.col("total_reason_start")).round(3).alias("transition_percentage"),
        (pl.col("total_reason_end") / pl.col("total")).round(3).alias("marginal_end_probability")
    ]).select(["hist_user_behavior_reason_start", "hist_user_behavior_reason_end", "count", "transition_percentage", "marginal_end_probability"])
    .sort("hist_user_behavior_reason_start", "hist_user_behavior_reason_end") 
)

transition_stats

hist_user_behavior_reason_start,hist_user_behavior_reason_end,count,transition_percentage,marginal_end_probability
str,str,u32,f64,f64
"""appload""","""backbtn""",126,0.065,0.078
"""appload""","""endplay""",408,0.21,0.088
"""appload""","""fwdbtn""",585,0.301,0.485
"""appload""","""logout""",12,0.006,0.003
"""appload""","""remote""",6,0.003,0.002
"""appload""","""trackdone""",805,0.415,0.344
"""backbtn""","""backbtn""",7121,0.546,0.078
"""backbtn""","""endplay""",468,0.036,0.088
"""backbtn""","""fwdbtn""",2378,0.182,0.485


#### Check if the reason that a song finished is different from reason a song start

In [28]:
df_sessions = (
    df_sessions
    .with_columns([
        # Shift the start reason to the next row within the session
        pl.col("hist_user_behavior_reason_start")
          .shift(-1)
          .over("session_id")
          .fill_null(pl.col("hist_user_behavior_reason_end"))
          .alias("hist_user_behavior_reason_start_next"),
    ])
    .with_columns([
        # Create a dummy column indicating if end ≠ next start
        (pl.col("hist_user_behavior_reason_end") != pl.col("hist_user_behavior_reason_start_next"))
          .alias("reason_end_differs_from_next_start")
    ])
)

In [29]:
df_sessions.group_by("reason_end_differs_from_next_start").agg(pl.len())

reason_end_differs_from_next_start,len
bool,u32
false,151910
true,15970


## Clustering

### Grouping the data

#### Summarise categorical Columns

In [6]:
# List of columns you want counts for
categorical_cols = [
    "context_type",
    "hist_user_behavior_reason_start",
    "hist_user_behavior_reason_end",
]

categorical_cols = {
    "context_type" : "context_type",
    "hist_user_behavior_reason_start": "reason_start",
    "hist_user_behavior_reason_end": "reason_end"
}

# Function to process each column and return a pivoted count table
def get_column_counts(df, col, prefix):
    pivoted = (
        df
        .group_by(["session_id", col])
        .agg(pl.len().alias(f"count_{col}"))
        .pivot(values=f"count_{col}", index="session_id", on=col)
        .fill_null(0)
    )

    pivoted = pivoted.rename({
        old: f"{prefix}_{old}" for old in pivoted.columns if old != "session_id"
    })

    return pivoted

# Create a list of pivoted count tables for each categorical column
categorical_counts = [get_column_counts(df_sessions, col, prefix) for col, prefix in categorical_cols.items()]


# Merge all count tables on 'session_id'
categorical_counts = reduce(lambda left, right: left.join(right, on="session_id", how="left"), categorical_counts)
categorical_counts = (
    categorical_counts
    .join(
        df_sessions
        .select(['session_id', "session_length"])
        .unique(),
        on="session_id", 
        how="left"
    ) 
)
for c in [col for col in categorical_counts.columns if col not in ['session_id', 'session_length']]:
        categorical_counts = categorical_counts.with_columns(
            (pl.col(c) / pl.col("session_length")).alias(c)
        )
categorical_counts = categorical_counts.drop("session_length")
categorical_counts.head(2)

session_id,context_type_editorial_playlist,context_type_user_collection,context_type_radio,context_type_catalog,context_type_personalized_playlist,context_type_charts,reason_start_clickrow,reason_start_fwdbtn,reason_start_backbtn,reason_start_appload,reason_start_trackdone,reason_start_playbtn,reason_start_trackerror,reason_start_remote,reason_start_endplay,reason_end_remote,reason_end_backbtn,reason_end_endplay,reason_end_trackdone,reason_end_fwdbtn,reason_end_logout,reason_end_clickrow
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0_06e1ddae-91fd-49ed-bce0-5dd2…",1.0,0.0,0.0,0.0,0.0,0.0,0.8,0.1,0.0,0.05,0.05,0.0,0.0,0.0,0.0,0.0,0.0,0.85,0.1,0.05,0.0,0.0
"""0_063da342-6fe4-42bc-8f17-c445…",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.1,0.3,0.0,0.0,0.0,0.0,0.0,0.0,0.1,0.3,0.6,0.0,0.0


#### Create Summary dataframe where 1 row is one session

In [7]:
df_summary = (
    df_sessions
    .group_by("session_id")
    .agg([
        pl.col("session_length").max().alias("session_length"),
        pl.col("premium").first(),
        (pl.col("context_switch").sum() / pl.col("session_length").max()).alias("context_switch_ratio"),
        (pl.col("hist_user_behavior_n_seekfwd").sum() / pl.col("session_length").max()).alias("seekfwd_ratio"),
        (pl.col("hist_user_behavior_n_seekback").sum() / pl.col("session_length").max()).alias("seekback_ratio"),
        ((pl.col("hist_user_behavior_n_seekback") + pl.col("hist_user_behavior_n_seekfwd")).sum() / pl.col("session_length").max()).alias("skip_ratio"),
        (pl.col("hist_user_behavior_is_shuffle").sum() / pl.col("session_length").max()).alias("shuffle_ratio"),
        pl.col("song_completion").mean().alias("avg_song_completion"),
        (pl.col("not_skipped").sum() / pl.col("session_length").max()).alias("perc_completed_song"),
        pl.col("pause_length_before_play").mean().alias("avg_pause_length_before_play"),
        pl.col("is_weekend").first().alias("is_weekend"),
        pl.col("season").first().alias("season"),
        pl.col("time_of_day").first().alias("time_of_day"),
    ])
    .join(
        categorical_counts,
        on="session_id",
        how='left'
    )
        
)

df_summary.head(1)

session_id,session_length,premium,context_switch_ratio,seekfwd_ratio,seekback_ratio,skip_ratio,shuffle_ratio,avg_song_completion,perc_completed_song,avg_pause_length_before_play,is_weekend,season,time_of_day,context_type_editorial_playlist,context_type_user_collection,context_type_radio,context_type_catalog,context_type_personalized_playlist,context_type_charts,reason_start_clickrow,reason_start_fwdbtn,reason_start_backbtn,reason_start_appload,reason_start_trackdone,reason_start_playbtn,reason_start_trackerror,reason_start_remote,reason_start_endplay,reason_end_remote,reason_end_backbtn,reason_end_endplay,reason_end_trackdone,reason_end_fwdbtn,reason_end_logout,reason_end_clickrow
str,i64,bool,f64,f64,f64,f64,f64,f64,f64,f64,i8,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""0_026ad0ff-8be6-4d15-b9bc-ccbd…",16,true,0.0625,0.0,0.0,0.0,0.0,0.5,0.25,0.3125,1,"""summer""","""night""",0.0,0.875,0.0,0.125,0.0,0.0,0.0625,0.5625,0.0625,0.0625,0.25,0.0,0.0,0.0,0.0,0.0,0.0625,0.0625,0.25,0.625,0.0,0.0


In [46]:
non_session_related_variables = ["premium", "is_weekend", "season", "time_of_day"]

In [15]:
(
    df_summary.
    group_by("premium")
    .agg([
    pl.col("perc_completed_song").mean()
    ])
)

premium,perc_completed_song
bool,f64
true,0.339423
false,0.34099


#### KMEANS